In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [22]:
import torch
import torch.nn as nn

class SevenLayerNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SevenLayerNN, self).__init__()

        self.fc1 = nn.Linear(input_size, 128)
        self.bn1 = nn.BatchNorm1d(128)

        self.fc2 = nn.Linear(128, 128)
        self.bn2 = nn.BatchNorm1d(128)

        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)

        self.fc4 = nn.Linear(64, 64)
        self.bn4 = nn.BatchNorm1d(64)

        self.fc5 = nn.Linear(64, 32)
        self.bn5 = nn.BatchNorm1d(32)

        self.fc6 = nn.Linear(32, 16)
        self.bn6 = nn.BatchNorm1d(16)

        self.fc7 = nn.Linear(16, num_classes)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = self.dropout(self.relu(self.bn1(self.fc1(x))))
        x = self.dropout(self.relu(self.bn2(self.fc2(x))))
        x = self.dropout(self.relu(self.bn3(self.fc3(x))))
        x = self.dropout(self.relu(self.bn4(self.fc4(x))))
        x = self.dropout(self.relu(self.bn5(self.fc5(x))))
        x = self.dropout(self.relu(self.bn6(self.fc6(x))))
        x = self.fc7(x)
        return x


In [23]:
#Dummy data

X=torch.randn(1000, 20)
y=torch.randint(0,2,(1000,))

dataset=TensorDataset(X,y)
dataloader=DataLoader(dataset, batch_size=32, shuffle=True)

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SevenLayerNN(input_size=20, num_classes=2).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [26]:
epochs = 10

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(dataloader):.4f}")


Epoch [1/10], Loss: 0.6969
Epoch [2/10], Loss: 0.6972
Epoch [3/10], Loss: 0.6985
Epoch [4/10], Loss: 0.6936
Epoch [5/10], Loss: 0.6966
Epoch [6/10], Loss: 0.7002
Epoch [7/10], Loss: 0.6933
Epoch [8/10], Loss: 0.6955
Epoch [9/10], Loss: 0.6934
Epoch [10/10], Loss: 0.6951


In [27]:
from sklearn.metrics import classification_report

model.eval()

y_true = []
y_pred = []

with torch.no_grad():
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())


In [28]:
print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.50      1.00      0.67       503
           1       0.00      0.00      0.00       497

    accuracy                           0.50      1000
   macro avg       0.25      0.50      0.33      1000
weighted avg       0.25      0.50      0.34      1000



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
